In [7]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt
from gphelper import GPHelper

from importlib import reload

import warnings
warnings.filterwarnings('ignore')


from multiprocessing import Pool

Using matplotlib backend: Qt5Agg


In [ ]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    img[img<=0] = img[img>0].min()
    return img

def get_random_spheroids(num=10, with_ids=False):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    if with_ids:
        return list(zip(selected, sources))
    else:
        return sources

# Petrosian Helpers ----------------------------------------------------------
def one_over_eta(rs, fs, R):
    return ((np.pi*R**2) / (np.sum(fs[rs<=R]))) * fs[rs==R]

def petrosian_radius(rs, fs):
    #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
    r_candidates = r_candidates[rs<12.5]
    
    return rs[np.square(r_candidates-0.2).argmin()]

def petrosian_flux(rs, fs, R_p):
    return np.sum(fs[rs<=2*R_p])

def R_x(rs, fs, x):
    x /= 100
    sum_ratio = np.cumsum(fs)/np.sum(fs)
    return rs[np.square(sum_ratio-x).argmin()]

def petrosian_Re(R_50, R_90):
    P_3 = 8e-6
    P_4 = 8.47
    R_ratio = min(R_90/R_50, 3.8)
    return R_50 / (1 - (P_3 * (R_ratio)**P_4))

def get_rs_and_fs(img, src_map):
    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))
    
    rs = rs.flatten()
    fs = img.flatten()
    sorted_rs = np.argsort(rs)
    
    rs = rs[sorted_rs]
    fs = fs[sorted_rs]
    
    return rs, fs

def get_re(img, src_map):
    rs, fs = get_rs_and_fs(img, src_map)
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx], rs, fs

def get_re(rsfs):
    rs, fs = rsfs
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx]

# Petrosian Helpers ----------------------------------------------------------

def denoise(img, segmap, img_id):
    noise_bank = img[segmap==0].flatten()
    other_source = np.logical_and(segmap!=0, segmap!=img_id)
    np.place(img, other_source, noise_bank)
    
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(img.newbyteorder('='), mask=segmap==img_id, bw=10, bh=10)
    img = img-bkg
    
    return img

# https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2.html
def sersic(x):
    def b(n):
        return 1.9992*n-0.3271

    def I(r):
        """Assuming that I_e=1.0 and that R_e=1.0"""
        n = 4
        return np.exp(-b(n) * (np.power(r, 1/n)  - 1))

    return I(x)

In [ ]:
_id, (img, segmap, img_id) = get_random_spheroids(num=1, with_ids=True)[0]
img = denoise(img, segmap, img_id)
plt.title(_id)
plt.imshow(img, cmap='gray')


In [ ]:
rs, fs = get_rs_and_fs(img, segmap==img_id)

str = 'r, I(r)\n'
for r, f in zip(rs, fs):
    str += f'{r},{f}\n'
    
with open('ascii_rsfs.csv', 'w') as f:
    f.write(str)

In [ ]:
rs, fs = [], []
with open('ascii_rsfs.csv', 'r') as f:
    for line in f:
        if 'I(r)' in line:
            continue
        nums = line.split(' ')
        rs.append(float(nums[0]))
        fs.append(float(nums[1]))
        
rs = np.array(rs)
fs = np.array(fs)

In [ ]:
plt.plot(rs, fs)

In [ ]:
%%time
dR=0.05*(rs.max()-rs.min())
print(dR)
f_smooth = dt.loessc_p(rs,fs,dx=dR, pnum=2)

In [ ]:
plt.plot(rs, fs)
plt.plot(rs, f_smooth)

In [ ]:
(f_smooth<0).sum()

In [ ]:
num_interp = 100
interp_rs = np.linspace(rs.min(), rs.max(), num_interp)
interp_fs = np.interp(interp_rs, rs, f_smooth)
interp_fs[interp_fs<0] = 0

plt.plot(interp_rs, interp_fs)

In [ ]:
L = np.zeros(num_interp)
L[0] = np.pi*interp_rs[0]**2  * interp_fs[0]
for i in range(1,num_interp,1):
    L[i] = L[i-1] + np.pi*(interp_rs[i]**2 - interp_rs[i-1]**2)*interp_fs[i]

In [ ]:
plt.plot(interp_rs, L)

In [ ]:
A = np.pi * interp_rs**2

In [ ]:
etas = interp_fs*A/L
plt.plot(interp_rs, etas)
plt.xlim([0,10])

In [ ]:
eta_idx = np.square(etas[interp_rs<10]-0.2).argmin()
interp_rs[eta_idx]

In [ ]:
etas[eta_idx]

In [4]:
spheroids = get_random_spheroids(num=-1)

In [9]:
%%time
#_ = process_img(spheroids[6])

processed = list(map(process_img, spheroids[:]))

#with Pool(5) as p:
#    processed = list(p.map(process_img, spheroids[:1]))

Re:8.837181945398621
Re:8.049487779005924
Re:8.970942680473359
Re:8.92059031934131
Re:9.989186613650642
Re:10.748041174697056
Re:11.889133394871063
Re:12.079726380228742
Re:11.67269567175413
Re:11.224715863578028
Re:11.619213225557777
Re:11.992807728359166
Re:10.857175386713134
Re:9.5804323753443
Re:12.077573114954964
Re:8.561424647605708
Re:12.315888713447194
Re:12.261443459121516
Re:7.842838037450152
Re:10.888760871308005
Re:12.405310243974213
Re:8.908785222375245
Re:11.636728330643779
Re:11.802227027714117
Re:8.383205439113276
Re:10.139627103416789
Re:9.280333514919121
Re:12.08782614215171
Re:8.577807799682459
Re:9.794707544140788
Re:9.244216401882133
Re:7.9324991619972804
Re:11.903890534629596
Re:7.174705932262391
Re:12.546928576535358
Re:8.687106758202136
Re:8.679735814898956
Re:11.880827302417119
Re:11.727048478007514
Re:9.310069813420384
Re:7.6200553721785855
Re:8.048511857742271
Re:8.799033965815541
Re:11.713946281823803
Re:10.06806864114908
Re:9.07903245474498
Re:8.80603844839

Re:10.347545224197223
Re:10.021082036257058
Re:12.189584748647759
Re:12.116794654294695
Re:8.359704482879442
Re:12.472486545441186
Re:8.696163082563219
Re:10.452172152387407
Re:8.318321762974776
Re:8.045179929928086
Re:9.071693157237334
Re:9.165911335421425
Re:11.526024637144676
Re:9.796380192527485
Re:8.712923452899346
Re:6.934223326864721
Re:7.840645411492026
Re:12.12036929403716
Re:12.433125194427763
Re:10.366870582125294
Re:7.913513842644886
Re:8.092212658307297
Re:11.48955602420695
Re:12.310721563954091
Re:10.351666374772343
Re:11.431214407590204
Re:10.334169192270798
Re:11.756736355219688
Re:8.241594805710836
Re:12.011793551775115
Re:8.511334632108191
Re:8.289943297131307
Re:9.002735848751078
Re:11.660296322579583
Re:7.614890393081186
Re:7.832270400094222
Re:10.23461591605676
Re:9.747145120531005
Re:9.092085544991926
Re:12.196160370527135
Re:9.589162470947015
Re:12.043582221373857
Re:10.976007286189533
Re:11.696615696278272
Re:8.85999594183707
Re:9.166994559207495
Re:11.337886375

In [10]:
res = [p[2] for p in processed]

In [11]:
plt.title('$R_e$')
plt.hist(res, bins=20)

(array([  2.,   1.,  12.,  18.,  29.,  36.,  30.,  49.,  31.,  19.,  20.,
         23.,  28.,  21.,  17.,  17.,  45.,  47.,  43.,  12.]),
 array([  6.38189199,   6.69936569,   7.01683939,   7.3343131 ,
          7.6517868 ,   7.9692605 ,   8.2867342 ,   8.60420791,
          8.92168161,   9.23915531,   9.55662901,   9.87410272,
         10.19157642,  10.50905012,  10.82652382,  11.14399753,
         11.46147123,  11.77894493,  12.09641863,  12.41389234,  12.73136604]),
 <a list of 20 Patch objects>)

In [8]:
def get_random_spheroids(num=10, with_ids=False):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    if with_ids:
        return list(zip(selected, sources))
    else:
        return sources

def process_img(args):
    
    def denoise(img, segmap, img_id):
        noise_bank = img[segmap==0].flatten()
        other_source = np.logical_and(segmap!=0, segmap!=img_id)
        np.place(img, other_source, noise_bank)

        sep_arr = img.byteswap().newbyteorder('N')
        #print(sep_arr.dtype.byteorder)
        bkg = sep.Background(sep_arr, mask=segmap==img_id, bw=10, bh=10)
        img = img-bkg

        return img
    
    def get_rs_and_fs(img, src_map):
        cx, cy = it.img_center(img, src_map)
        xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
        rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))

        rs = rs.flatten()
        fs = img.flatten()
        sorted_rs = np.argsort(rs)

        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        return rs, fs
    
    img, segmap, img_id = args
    src_map = segmap==img_id
    img = denoise(img, segmap, img_id)
    
    #plt.figure()
    #plt.imshow(img, cmap='gray')
    
    rs, fs = get_rs_and_fs(img, src_map)

    
    #plt.figure()
    #plt.plot(rs, fs)
    
    
    dr = 0.05*(rs.max()-rs.min())
    #print(fs[:10])
    fs = dt.loessc_p(rs, fs, dr, pnum=1)
    #print(fs[:10])
    fs[fs<0]=0
    
    #plt.figure()
    #plt.plot(rs, fs)
    
    num_interp = 100
    _rs = np.linspace(rs.min(), rs.max(), num_interp)
    fs = np.interp(_rs, rs, fs)
    rs = _rs
    
    #plt.figure()
    #plt.plot(rs, fs)
    
    L = np.zeros(num_interp)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,num_interp,1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*fs[i]

    A = np.pi * rs**2
    #print(rs, fs)
    etas = fs*A/L
    r_lim = rs<10.0
    R_p = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    #print(etas)
    
    #plt.figure()
    #plt.plot(etas[r_lim], rs[r_lim], '.')
    #plt.vlines([0.2], 0, 12.5)
    #print(np.flipud(etas[r_lim]), rs[r_lim])
    #print(f'R_p:{R_p}')
    
    #plt.figure()
    #plt.plot(rs, etas)
    
    Fp_idx = np.square(rs - 2*R_p).argmin()
    #print(Fp_idx)
    
    F_p = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    
    
    #plt.figure()
    #plt.plot(F_p, rs[:Fp_idx])
    #print(len(F_p), len(rs[:Fp_idx]))
    
    R_50 = np.interp(.5, F_p, rs[:Fp_idx])
    R_90 = np.interp(.9, F_p, rs[:Fp_idx])
    #print(F_p, rs[:Fp_idx])
    #print(f'R_50:{R_50} R_90:{R_90}')
    
    
    P3, P4 = 8e-6, 8.47
    re = R_50 / (1 - P3 * (R_90 / R_50)**P4)
    ie = np.interp(re, rs, fs)
    print(f'Re:{re}')
    return (rs, fs, re, ie)

In [ ]:
reload(dt)

re_max_ratio = []

res = []
ies = []
normed_rs = []
normed_fs = []
count = 0

small_res = []

for img, segmap, img_id in spheroids:
    count += 1
    
    print(count/len(_spheroids), end='\r')

    #background subtract images
    src_map = segmap==img_id
    
    img = denoise(img, segmap, img_id)
    
    # get the sorted r vals and i vals
    rs, fs = get_rs_and_fs(img, src_map)
    
    # smooth the fs using loess
    #fs = dt.loessc(rs, fs, .60)
    dR = 0.05*(R.max()-R.min())
    fs = dt.loessc_p(rs, fs, dR, pnum=2)
    #print(len(rs), len(fs))
    #fs = lowess(fs, rs)
    #print(len(rs), len(fs))
    
    # measure petrosian re 
    re = get_re((rs,fs))

    if re<3:
        small_res.append((img_id, re))
    
    re_idx = np.where(rs==re)[0][0] 
    
    ie = fs[re_idx]
    
    #sum_to_re = fs[rs<=re_idx].sum()
    #src_L = sum_to_re*2
    #img_L = fs.sum()
    #if src_L>img_L:
    #    re_max_ratio.append(sum_to_re/img_L)
    #    continue
    
    #max_r = np.square(np.cumsum(fs)-src_L).argmin()
    
    #max_r = rs[max_r]
    
    #fs = fs
    #rs = rs
    pos = lambda a: a if (a<=0).sum()==0 else a + ap.abs(a.min()) + 1e-3
    normed_rs.append(pos(rs))
    normed_fs.append(fs)
    ies.append(ie)
    res.append(re)    

In [62]:
plt.title('Effective Radius')
plt.xlabel('$R_e$(Pixels)')
plt.hist(res, bins=50)

(array([  1.,   0.,   1.,   0.,   1.,   2.,   7.,   4.,   4.,  13.,  11.,
         10.,  18.,  16.,  10.,  18.,   7.,  17.,  18.,  19.,   7.,  17.,
          9.,   7.,  10.,   7.,   7.,  11.,   9.,   9.,  18.,   8.,   7.,
          7.,   9.,   8.,   8.,   2.,   5.,  11.,  10.,  27.,  15.,  16.,
         24.,  18.,  17.,  10.,   7.,   3.]),
 array([  6.38189199,   6.50888147,   6.63587095,   6.76286043,
          6.88984991,   7.01683939,   7.14382887,   7.27081835,
          7.39780784,   7.52479732,   7.6517868 ,   7.77877628,
          7.90576576,   8.03275524,   8.15974472,   8.2867342 ,
          8.41372368,   8.54071316,   8.66770265,   8.79469213,
          8.92168161,   9.04867109,   9.17566057,   9.30265005,
          9.42963953,   9.55662901,   9.68361849,   9.81060797,
          9.93759746,  10.06458694,  10.19157642,  10.3185659 ,
         10.44555538,  10.57254486,  10.69953434,  10.82652382,
         10.9535133 ,  11.08050278,  11.20749227,  11.33448175,
         11.461471

In [ ]:
stat_vals.shape

In [ ]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])
#stat_vals = np.dstack(stat_vals)[0,...]
f_mean = np.log10(np.nanmedian(stat_vals, axis=0))
f_16 = np.log10(np.nanpercentile(stat_vals, 16, axis=0))
f_84 = np.log10(np.nanpercentile(stat_vals, 84, axis=0))
    
plt.plot(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [12]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])

stat_vals = []

plt.figure()
plt.title('Normalized Surface Brightness')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$ (Log)')

for rs, fs, re, ie in processed:
    color = 'b'
    if np.sum(fs<=0) > 0:
        fs = fs + np.abs(fs.min()) + 1e-3
        ie = np.interp(re, rs, fs)
    
    r = rs/re
    f = fs/ie
    plt.plot(r, np.log10(f), color='k', alpha=.05)
    stat_vals.append(np.interp(interped_marks, r, f, left=np.nan, right=np.nan))
    

stat_vals = np.log10(np.dstack(stat_vals)[0,...])
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)
    
plt.plot(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [35]:
with open('source_rs.csv', 'w') as f:
    f.write(','.join([str(v) for v in interped_marks]) + '\n')
    for i in range(stat_vals.shape[1]):
        f.write(','.join([str(v) for v in stat_vals[:,i]]) + '\n')
    f.write('---next is 16th,median,84th\n')
    for a in [f_16,f_mean,f_84]:
        f.write(','.join([str(v) for v in a]) + '\n')    

In [13]:
stat_vals.shape

(1000, 500)

In [ ]:
plt.scatter(res, ies)
plt.xlabel('$R_e$')
plt.ylabel('$I_e$')
plt.title('$R_e$ $I_e$ Correlation')
np.corrcoef(res, ies)

In [65]:
valid_points = ~np.isnan(stat_vals).all(axis=1)

X = interped_marks[valid_points]
Y = f_mean[valid_points]
a = f_84[valid_points] - f_mean[valid_points]

vals = {'x':dt._nmpy_encode(X),
        'y':dt._nmpy_encode(Y),
        'a':dt._nmpy_encode(a)}
with open('vals_for_gp.json', 'w') as f:
    json.dump(vals,f)

In [18]:
with open('vals_for_gp.json', 'r') as f:
    vals = json.load(f)
    
X = dt._nmpy_decode(vals['x'])
Y = dt._nmpy_decode(vals['y'])
a = dt._nmpy_decode(vals['a'])

print(X.shape, Y.shape, a.shape)

(464,) (464,) (464,)


In [20]:
upto=5

In [139]:
# truncate lines from 5re and on
upto = 5
_X = X[X<=upto]
_Y = Y[X<=upto]
_a = a[X<=upto]
tmp = _Y-_a




print(np.isnan(_X).sum(), np.isnan(_Y).sum(), np.isnan(_a).sum())
split_idx = 0#np.argmin(np.diff(_a))
split_val = _X[split_idx]

# fit and pad line to smooth
plt.plot(_X[split_idx:],_Y[split_idx:], label='Measured Values')
plt.fill_between(_X[split_idx:], tmp[split_idx:], _Y[split_idx:]+_a[split_idx:], alpha=0.2)

_X = _X[split_idx:]
_Y = _Y[split_idx:]
_a = _a[split_idx:]

_X, _Y = dt.pad_line(_X, _Y, 10, 10, append=False)
_, _a = dt.pad_line(_X, _a, 10, 10, append=False)

plt.plot(_X[_X<split_val], _Y[_X<split_val], '--', label='Linear Fit Values')
plt.fill_between(_X[_X<split_val], _Y[_X<split_val]-_a[_X<split_val], _Y[_X<split_val]+_a[_X<split_val], alpha=0.2)

plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e (Log)$')
plt.title('Measured Surface Brightnes W/Linear Fit Projection')
plt.legend()

0 0 0


In [140]:
gp = GPHelper()
gp.fit(_X[:,np.newaxis], _Y, _a, optimize='both')
_x, std = gp.predict(_X[:, np.newaxis], return_std=True)

In [141]:
plt.plot(_X,_Y, label='GP Fit')
plt.fill_between(_X, _Y-std, _Y+std, alpha=0.2)
plt.legend()

In [22]:
gp = GPHelper(restore_file='gp.json')

In [21]:
test_vals = np.concatenate([np.linspace(0.01, 1, 10, endpoint=False), np.linspace(1, upto, upto*10)])

In [23]:
#samples = gp.sample(test_vals[:,np.newaxis], num_samples=500)
samples = []
for i in range(500):
    print(i/500)
    samples.append(gp.sample(test_vals[:,np.newaxis]))

0.0
0.002ng Samples...  
0.004ng Samples     
0.006ng Samples     
0.008ng Samples     
0.01ing Samples.    
0.012ng Samples..   
0.014ng Samples     
0.016ng Samples...  
0.018ng Samples.    
0.02ing Samples.    
0.022ng Samples     
0.024ng Samples.    
0.026ng Samples...  
0.028ng Samples     
0.03ing Samples     
0.032ng Samples     
0.034ng Samples..   
0.036ng Samples...  
0.038ng Samples     
0.04ing Samples...  
0.042ng Samples..   
0.044ng Samples...  
0.046ng Samples...  
0.048ng Samples...  
0.05ing Samples..   
0.052ng Samples     
0.054ng Samples..   
0.056ng Samples     
0.058ng Samples..   
0.06ing Samples     
0.062ng Samples.    
0.064ng Samples...  
0.066ng Samples.    
0.068ng Samples...  
0.07ing Samples.    
0.072ng Samples     
0.074ng Samples     
0.076ng Samples     
0.078ng Samples     
0.08ing Samples..   
0.082ng Samples.    
0.084ng Samples..   
0.086ng Samples.    
0.088ng Samples     
0.09ing Samples...  
0.092ng Samples..   
0.094ng Samples.    
0.096ng S

0.784ng Samples...  
0.786ng Samples..   
0.788ng Samples     
0.79ing Samples.    
0.792ng Samples...  
0.794ng Samples     
0.796ng Samples     
0.798ng Samples..   
0.8wing Samples     
0.802ng Samples.    
0.804ng Samples.    
0.806ng Samples...  
0.808ng Samples.    
0.81ing Samples...  
0.812ng Samples..   
0.814ng Samples.    
0.816ng Samples     
0.818ng Samples     
0.82ing Samples..   
0.822ng Samples     
0.824ng Samples...  
0.826ng Samples..   
0.828ng Samples     
0.83ing Samples.    
0.832ng Samples     
0.834ng Samples..   
0.836ng Samples     
0.838ng Samples.    
0.84ing Samples.    
0.842ng Samples.    
0.844ng Samples     
0.846ng Samples...  
0.848ng Samples.    
0.85ing Samples     
0.852ng Samples..   
0.854ng Samples.    
0.856ng Samples...  
0.858ng Samples..   
0.86ing Samples...  
0.862ng Samples...  
0.864ng Samples...  
0.866ng Samples...  
0.868ng Samples...  
0.87ing Samples     
0.872ng Samples.    
0.874ng Samples.    
0.876ng Samples..   
0.878ng Sampl

In [24]:
samples = np.stack(samples)

In [37]:
np.percentile(samples, 16, axis=0).shape

(60,)

In [39]:
with open('gp_vals.csv', 'w') as f:
    f.write(','.join([str(v) for v in test_vals]) + '\n')
    
    for i in range(500):
        f.write(','.join([str(v) for v in samples[i,:]]) + '\n')
    
    f.write('---next is 16th,median,84th\n')
    for i in [16, 50, 84]:
        f.write(','.join([str(v) for v in np.percentile(samples, i, axis=0)]))

In [158]:
plt.title('GP 500 Samples')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$ (Log)')

for i in range(samples.shape[0]):
    plt.plot(test_vals, samples[i,:,0], color='b', alpha=0.2)

In [40]:
x = test_vals
gp_res = []
for (_, _, re, ie), fs in zip(processed, list(samples[:,:])):
    fs = 10**fs * ie
    rs = x * re

    L = np.zeros_like(fs)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,len(fs),1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*fs[i]

    A = np.pi * rs**2
    etas = fs*A/L
    r_lim = rs<10.0
    
    R_p = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    Fp_idx = np.square(rs - 2*R_p).argmin()
    F_p = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    R_50 = np.interp(.5, F_p, rs[:Fp_idx])
    R_90 = np.interp(.9, F_p, rs[:Fp_idx])
    
    
    P3, P4 = 8e-6, 8.47
    re = R_50 / (1 - P3 * (R_90 / R_50)**P4)
    ie = np.interp(re, rs, fs)
    
    gp_res.append(re)

In [84]:
print(len(res), len(gp_res))
plt.hist(res, bins=40, alpha=0.5, label='Data')
#plt.hist(new_res, bins=40, alpha=0.5, label='Line Measured')
#plt.figure()
plt.hist(gp_res, bins=40, alpha=0.5, label='GP')
plt.xlabel("Effective Radius")
plt.legend()

500 500


In [75]:
xs, ys = np.meshgrid(np.arange(84), np.arange(84))
cxy = 42
rs = np.sqrt((xs-cxy)**2 + (ys-cxy)**2)

s = samples[5,:]
_, _, re, ie = processed[35]
rs = rs/re
fs = s*ie


fs = np.interp(rs.flatten(), test_vals, fs).reshape([84,84])
plt.imshow(fs, cmap='gray')

In [76]:
img, segmap, img_id = spheroids[1]

noise = img[segmap==0]

frame = np.zeros([84,84])
np.place(frame, np.ones_like(frame), noise)
np.random.shuffle(frame)
plt.imshow(frame, cmap='gray')

In [81]:
plt.imshow(fs+(frame*0.06), cmap='gray')

In [82]:
fits.PrimaryHDU(frame).writeto('noise_bank.fits')
fits.PrimaryHDU(fs).writeto('gp_img.fits')
fits.PrimaryHDU(fs+(frame*0.06)).writeto('gp_with_noise.fits')